# RNN
---
> https://pytorch.org/docs/stable/generated/torch.nn.RNN.html#torch.nn.RNN
torch.nn.RNN()

### 1. RNN 파라미터
---

![](./%EC%9D%B4%EB%AF%B8%EC%A7%80/image_27.png)

1. 파라미터

    1. input_size 

        입력해 주는 특성 값의 개수 입니다. 만약 feature의 개수가 1이라면 input_size=1, 입력 feature 개수가 7개면 input_size=7을 입력합니다.

    2. hidden_size

        hidden state의 개수를 지정합니다. 보통 arbitrary 합니다.

    3. num_layers

        RNN 레이어를 겹겹이 쌓아올릴 수 있습니다. RNN 레이어를 쌓아 올리는 것을 stacking RNN이라고도 합니다. 만약, 2개층을 겹겹이 쌓아올린다면 num_layers=2 로 설정하면 됩니다. 기본 값: 1

    4. nonlinearity

        사용할 비선형성입니다. 'tanh' 또는 'relu'일 수 있습니다. 기본값: 'tanh'

    5. bias

        False인 경우 레이어는 바이어스 가중치 b_ih와 b_hh를 사용하지 않습니다. 기본값: True

    6. batch_first

        입력으로 받는 데이터의 shape중 첫 번째 차원을 batch로 간주할 것인지를 설정합니다. 일반적으로 pytorch에서 데이터 전처리시 batch를 첫번째 차원으로 지정하기 때문에 많은 케이스에서 batch_firtst=True 로 지정함을 볼 수 있습니다. True이면 입력 및 출력 텐서가 (seq, batch, feature) 대신 (batch, seq, feature)로 제공됩니다. 이는 hidden 또는 cell state에는 적용되지 않습니다. 기본값: False

    7. dropout

        0이 아닌 경우, 마지막 레이어를 제외한 각 RNN 레이어의 출력에 드롭아웃 레이어를 도입하며, 드롭아웃 확률은 dropout과 같습니다. 기본값: 0
        
    8. bidirectional

         True이면 양방향 RNN이 됩니다. 기본값: False

### 2. 사용 예제
---

``` py
>>> rnn = nn.RNN(10, 20, 2)
>>> input = torch.randn(5, 3, 10)
>>> h0 = torch.randn(2, 3, 20)
>>> output, hn = rnn(input, h0)
```

In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F

In [2]:
input_size = 1  # 입력 데이터 특성 차원
hidden_dim = 10 # hidden state 차원
n_layers = 2    # stacking layer 개수

rnn = nn.RNN(input_size, hidden_dim, n_layers, batch_first=True)
rnn

RNN(1, 10, num_layers=2, batch_first=True)

In [7]:
# 20개의 시퀀스 생성
seq_length = 20

time_steps = np.linspace(0, np.pi, seq_length*input_size)
print(time_steps.shape)
# 출력
# (20,)

data = np.sin(time_steps)
data.resize((seq_length, 1))
print(data.shape)
# 출력
# (20, 1)

# 배치 차원 추가(0번째)
input_data = torch.Tensor(data).unsqueeze(0)
print('Input size: ', input_data.shape)
# Input size:  torch.Size([1, 20, 1])



(20,)
(20, 1)
Input size:  torch.Size([1, 20, 1])


In [8]:
# RNN 출력(output, hidden_state)
output, hidden_state = rnn(input_data, None)
# rnn output, hidden_state 차원

print('Output: ', output.size())
print('Hidden State: ', hidden_state.size())

Output:  torch.Size([1, 20, 10])
Hidden State:  torch.Size([2, 1, 10])


In [9]:
rnn = nn.RNN(10, 20, 2) 
input = torch.randn(5, 3, 10) # 기본적으로 (seq, batch, feature)
h0 = torch.randn(2, 3, 20) # 기본적으로 (seq, batch, feature)

output, hn = rnn(input, h0)
print(output.shape)
print(hn.shape)

torch.Size([5, 3, 20])
torch.Size([2, 3, 20])


### 3. RNN 만들어보기
---
아래 코드는 어떻게 작동되는 걸까?

In [10]:
rnn = nn.RNN(10, 20, 2) 
input = torch.randn(5, 3, 10) # 기본적으로 (seq, batch, feature)
h0 = torch.randn(2, 3, 20) # 기본적으로 (seq, batch, feature)

output, hn = rnn(input, h0)
print(output.shape) # [5, 3, 20]
print(hn.shape) # [2, 3, 20]

torch.Size([5, 3, 20])
torch.Size([2, 3, 20])


일반적으로 3차원의 행렬을 계산하는 방법은 없다. 우리가 합성곱을 할때도 img2col이라는 특수 함수를 이용해서 다차원의 행렬을 2차원으로 바꿔서 행렬 계산을 했던것을 떠올려보라.

그럼 RNN은 어떻게 구현이 되었을까? 파이토치의 RNN과 비슷하게 구현해보겠다.

![](./%EC%9D%B4%EB%AF%B8%EC%A7%80/image_28.png)

In [11]:
class RNN_Base(nn.Module):
    def __init__(self, input_size, hidden_size):
        super().__init__()
        self.W_x = nn.Linear(input_size, hidden_size) 
        self.W_h = nn.Linear(input_size, hidden_size) 
        self.h = None # 은닉 상태 벡터

    def reset_state(self):
        self.h = None # 은닉 상태를 초기화

    def forward(self, x):
        if self.h is None: #현재 가진 은닉 상태가 없다면?
            h_new = F.tanh(self.W_x(x)) # 그냥 입력값으로만 계산해서 첫번째 은닉 상태를 추출 tanh( x * W_x )
        else: # 은닉 상태가 있다면
            h_new = F.tanh(self.W_x(x) + self.W_h(self.h)) # tanh( (x * W_x) + (h * W_h + b) )
            self.h = h_new
        return h_new

### 4. RNN의 입력과 출력에 관해서
---

> 참고하면 좋은 링크 : https://wegonnamakeit.tistory.com/52

![](./%EC%9D%B4%EB%AF%B8%EC%A7%80/image_29.png)

N : 미니배치 크기

D : 입력 벡터 차원 수

H : 은닉 상태 차원 수

T : 길이가 T인 시계열 데이터 (간단히 말하자면 문장정도?)

F : bidirectional ( 양방향일경우 : 2, 단방향일경우 : 1 ) 

RNN에 들어가는 input은 총 2가지이다. input (위 사진에선 x<sub>t</sub>) 그리고 은닉상태인 h (위 사진에선 h<sub>t-1</sub>)

1. input의 형상 

    1. 배치가 없는 input의 형상 : (Sequence length, input_size) 

        간단히 표기하면 (T, D)

    2. 배치가 있으면서 batch_first=False 일때 input의 형상 : (Sequence length, batch_size, input_size)

        간단히 표기하면 (T, N, D)

    3. 배치가 있으면서 batch_first=True 일때 input의 형상 : (batch_size, Sequence length, input_size)

        간단히 표기하면 (N, T, D)

2. h의 형상

    1. 배치가 없을 경우 : ( bidirectional ( 양방향일경우 : 2, 단방향일경우 : 1 ) * num_layers, hidden_size)

        간단히 표기하면 (F * num_layers, H) 

        F = bidirectional ( 양방향일경우 : 2, 단방향일경우 : 1 ) * num_layers 인데 단방향일 경우 통과한 RNN의 횟수정도로 생각하면 편하다

        RNN을 겹겹이 쌓아 올렸다는 것은 다시 말해 밑의 사진처럼 쌓아 올린것(여러번 통과시킨 것)
        
        ![](./%EC%9D%B4%EB%AF%B8%EC%A7%80/image_33.png)
        
        즉, 단어가 RNN을 통과한 횟수라고 생각하면 편하다 (아니면 내가 통과하도록 설정한 레이어의 갯수정도)

        단방향일 경우 그저 (T, H) 인것 

        아마 추측하건데 h의 정보를 담고 있는것 같다, 무슨 뜻인지는 잘은 모르겠다....



    2. 배치가 있을 경우 : ( bidirectional ( 양방향일경우 : 2, 단방향일경우 : 1 ) * num_layers, batch_size, hidden_size)

        간단히 표기하면 (F * num_layers, N, H)

RNN에서 나오는 output은 총 2가지이다.

1. output의 형상

    1. 배치가 없는 output의 형상 : (Sequence length, bidirectional ( 양방향일경우 : 2, 단방향일경우 : 1 ) * num_layers * hidden_size) 

        간단히 표기하면 (T, F * H)

    2. 배치가 있으면서 batch_first=False 일때 output의 형상 : (Sequence length, batch_size, bidirectional ( 양방향일경우 : 2, 단방향일경우 : 1 ) * num_layers * hidden_size)

        간단히 표기하면 (T, N, F * H)

    3. 배치가 있으면서 batch_first=True 일때 output의 형상 : (batch_size, Sequence length, bidirectional ( 양방향일경우 : 2, 단방향일경우 : 1 ) * num_layers * hidden_size)

        간단히 표기하면 (N, T, F * H)

2. h의 형상

    1. 배치가 없을 경우 : ( bidirectional ( 양방향일경우 : 2, 단방향일경우 : 1 ) * num_layers, hidden_size)

        간단히 표기하면 (F * num_layers, H)

    2. 배치가 있을 경우 : ( bidirectional ( 양방향일경우 : 2, 단방향일경우 : 1 ) * num_layers, batch_size, hidden_size)

        간단히 표기하면 (F * num_layers, N, H)

### 5. Sequence length가 무엇일까?
---

![](./%EC%9D%B4%EB%AF%B8%EC%A7%80/image_30.png)
    
Sequence length는 Input data의 길이를 뜻한다. PyTorch에서는 값을 자동으로 계산하기 때문에 input_size 값만 잘 입력해주면 된다.

이거는 '밑바닥부터 시작하는 딥러닝 2'의 RNN과 TimeRNN의 차이라고 생각하면 편하다.

TimeRNN 계층은 RNN 계층을 T개 연결한 신경망이다. 

![](./%EC%9D%B4%EB%AF%B8%EC%A7%80/image_31.png)

![](./%EC%9D%B4%EB%AF%B8%EC%A7%80/image_32.png)
    

In [ ]:
class RNN_Base(nn.Module): # 딱 1개의 단어 벡터만 입력 가능
    def __init__(self, input_size, hidden_size):
        super().__init__()
        self.W_x = nn.Linear(input_size, hidden_size) 
        self.W_h = nn.Linear(input_size, hidden_size) 
        self.h = None # 은닉 상태 벡터

    def reset_state(self):
        self.h = None # 은닉 상태를 초기화

    def forward(self, x):
        if self.h is None: #현재 가진 은닉 상태가 없다면?
            h_new = F.tanh(self.W_x(x)) # 그냥 입력값으로만 계산해서 첫번째 은닉 상태를 추출 tanh( x * W_x )
        else: # 은닉 상태가 있다면
            h_new = F.tanh(self.W_x(x) + self.W_h(self.h)) # tanh( (x * W_x) + (h * W_h + b) )
            self.h = h_new
        return h_new

In [ ]:
class RNN(nn.Module): # T개의 시계열 데이터 입력 가능 
    def __init__(self, input_size, hidden_size):
        super().__init__()
        self.RNN = RNN_Base(input_size, hidden_size)
        self.hidden_size = hidden_size

    def reset_state(self):
        self.RNN.reset_state()

    def forward(self, xs, h): 
        N, T, D = xs.shape # (batch_size, Sequence length, input_size) 
        H = self.hidden_size
        # N : 미니배치 크기, T: 길이가 T인 시계열 데이터, D : 입력 벡터 차원 수, H : 은닉 상태 차원 수

        hs = torch.tensor.zeros(N,T,H) # 출력 hs

        for t in range(T): # RNN_Base은 단어 1개뿐이 입력이 안되니까 하나씩 꺼내서 입력하자
            h = self.RNN(xs[:,t,:]) 
            hs[:,t,:] = H

        return hs

### 6. 헷갈리네요... 왜 파이토치의 RNN은 2개를 반환하죠??
---
정확하게 무엇을 반환하는지 알아야 한다.

RNN 셀은 두 개의 입력을 리턴하는데, 첫번째 리턴값은 모든 시점(timesteps)의 은닉 상태들이며, 두번째 리턴값은 마지막 시점(timestep)의 은닉 상태입니다. 

![](../image/image_34.jpg)

![](../image/image_36.jpg)

![](../image/image_35.jpg)

이렇게 암기하면 된다!




In [1]:
rnn = nn.RNN(10, 20) 
input = torch.randn(5, 3, 10) # 기본적으로 (seq, batch, feature)
h0 = torch.randn(2, 3, 20) # 기본적으로 (seq, batch, feature)

output, hn = rnn(input, h0)
print(output.shape) # [5, 3, 20]
print(hn.shape) # [2, 3, 20]

NameError: name 'nn' is not defined